In [1]:
#libraries and data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import scipy.stats 
import random


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#Classes to process the data
class State(object):
  def __init__(self, state, new_cases_per_100k, dem_margin):
    self.state = state
    self.cases = new_cases_per_100k
    self.dem_margin = dem_margin
  
  def toString(self):
    return f"State: {self.state}, Average New Weekly Cases Per 100k: {self.cases}, Democratic Margin of Victory: {self.dem_margin}"

#Container class to hold the data
class Container(object):
  def __init__(self):
    self.states = []
  
  def add(self, state):
    self.states.append(state)
  
  def toString(self):
    string = ''
    for item in self.states:
      string += item.toString() + "\n"
    return string

class StateCovidData(object):
  def __init__(self, state):
    self.state = state
    self.cases = 0
    self.point_count = 0
    self.sum_of_cases = 0
  
  def add(self, new_cases_per_100k):
    self.point_count += 1
    self.sum_of_cases += new_cases_per_100k
    self.cases = self.sum_of_cases/self.point_count
    
  def toString(self):
    return f"{self.state}, Average New Weekly Cases Per 100k: {self.cases}"

: 

In [ ]:
#Covid Data 
cov_key = "2c3b686a09214915b1471a2f0df00d20"
cov_url = "https://api.covidactnow.org/v2/states.timeseries.csv?apiKey="+cov_key

#Create dataframe
df = pd.read_csv(cov_url)

: 

In [ ]:
covid_data = df.loc[:, ["date", "state", "metrics.weeklyCovidAdmissionsPer100k"]]

#Take the average of all the data for each state and add it to a dictionary with the state as the key
state_covid_data = {}

for row in covid_data.iterrows():
  #continue if nan 
  if pd.isna(row[1]["metrics.weeklyCovidAdmissionsPer100k"]):
    continue

  if row[1].state in state_covid_data:
    state_covid_data[row[1].state].add(row[1]["metrics.weeklyCovidAdmissionsPer100k"])
  else:
    state_covid_data[row[1].state] = StateCovidData(row[1].state)
    state_covid_data[row[1].state].add(row[1]["metrics.weeklyCovidAdmissionsPer100k"])


: 

In [ ]:

#Voting Data 
vote_data = pd.read_csv(r'../data/pop_vote2.csv')

#Make a dictionary with the states as keys and the democratic margin as values
vote_dict_names = dict(zip(vote_data['state'], vote_data['dem_this_margin']))

#Dict of abbreviations and full names
state_dict = {'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland', 'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina', 'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming'}
#Reverse the list to use the name to find the variables 
#inv_map = {v: k for k, v in my_map.items()}
state_dict = {v:k for k, v in state_dict.items()}

vote_dict_abbrev = {}

#Rename the states to their abrev
for item in vote_dict_names:
    try:
        abbrev = state_dict[item]
        vote_dict_abbrev[abbrev] = vote_dict_names[item]
    except:
        continue

: 

In [ ]:
#Put all the data into a container object
container = Container()

for item in vote_dict_abbrev:
    state_obj = State(state=item, dem_margin=vote_dict_abbrev[item], new_cases_per_100k=state_covid_data[item].cases)
    container.add(state_obj)

: 

In [ ]:
#Make a plot for the data 

# Make two arrays, one for the dem_margin_vic and one for the cases 
dem_margin = []
cases_per_100k = []

for item in container.states:
  if item.cases == None: 
    continue
  else: 
    cases_per_100k.append(item.cases)
    dem_margin.append(item.dem_margin)

dem_float_arr = []

for item in dem_margin:
   dem_float_arr.append(float(item.replace("%", "")))


: 

In [ ]:
# linear line     
cases = pd.DataFrame(cases_per_100k).values
dem_margin = pd.DataFrame(dem_float_arr).values

linear = LinearRegression()
linear.fit(cases, dem_margin)
y_pred = linear.predict(cases)

plt.figure(0)
plt.scatter(cases_per_100k, dem_float_arr)

# correlation
correlation, p_value_correlation = scipy.stats.pearsonr(cases_per_100k, dem_float_arr)
print(f"Correlation: {correlation}, P Value of Correlation: {p_value_correlation}")

# polyfit of 3rd degree, change the number in the polyfit function to change the degree of fit
fit, cov = np.polyfit(cases_per_100k, dem_float_arr, 1, cov=True)
x_fit = np.linspace(np.min(cases),np.max(cases))
poly_fit = np.polyval(fit,x_fit)
plt.plot(x_fit, poly_fit, color = 'green')
plt.figure(0)

plt.xlabel("Average New Weekly Cases Per 100k Residents Since 3/1/2020")
plt.ylabel("Democratic Margin Victory in the 2022 Election (%)")
plt.title("Covid Cases and Democratic Margin Victory in the 2020 Election")
plt.scatter(cases_per_100k, dem_float_arr, color='black')
plt.show()

: 

In [ ]:
# cross_validation
def cross_validation(remove_nums, dataSetX, dataSetY, iteration):
    slope_array = []
    for i in range (iteration):
        copy_X = dataSetX.copy()
        copy_Y = dataSetY.copy()
        remove_indexs = random.sample(range(len(copy_X)), remove_nums)
        for i in range(len(remove_indexs)):
            copy_X.pop(remove_indexs[i]-i) #Because the list is getting smaller, we need to subtract i from the index
            copy_Y.pop(remove_indexs[i]-i)  
        fit = np.polyfit(copy_X, copy_Y, 1) 
        slope_array.append(fit)
    return slope_array

: 

In [ ]:
slope_array = cross_validation(3, cases_per_100k, dem_float_arr, 10)
slope = []
intercept = []
for i in slope_array:
    slope.append(i[0])
    intercept.append(i[1])
    
mean_slope = np.mean(slope)
mean_intercept = np.mean(intercept)

std_intercept = np.std(intercept)
std_slope = np.std(slope)

print(f"Mean Slope: {mean_slope}, Mean Intercept: {mean_intercept}, Std Slope: {std_slope}, Std Intercept: {std_intercept}))")

n = 10000000
slope_hist = np.random.normal(mean_slope, std_slope, n)
intercept_hist = np.random.normal(mean_intercept, std_intercept, n)

bins = 500
plt.hist(slope_hist, bins=bins, color='green')
plt.title("Slope Histogram")
plt.show()
plt.hist(intercept_hist, bins=bins, color='red')
plt.title("Y-Intercept Histogram")
plt.show()
plt.figure(1)

x_fit = np.linspace(np.min(cases),np.max(cases))
fit = [mean_slope, mean_intercept]
poly_fit = np.polyval(fit,x_fit)
plt.plot(x_fit, poly_fit, color = 'red')

x_fit = np.linspace(np.min(cases),np.max(cases))
fit = [mean_slope, mean_intercept]
poly_fit = np.polyval(fit,x_fit)
plt.plot(x_fit, poly_fit, color = 'red')

plt.xlabel("Average New Weekly Cases Per 100k Residents Since 3/1/2020")
plt.ylabel("Democratic Margin Victory in the 2022 Election (%)")
plt.title("Covid Cases and Democratic Margin Victory in the 2020 Election")
plt.scatter(cases_per_100k, dem_float_arr, color='black')

: 

: 